# Tabular Playground - Jan 2021 - CatBoost Regression

# 1. Librerias a utilizar

In [1]:
import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
import matplotlib.pyplot as plt
import seaborn as sns

from pathlib import Path
input_path = Path('/kaggle/input/tabular-playground-series-jan-2021/')


from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from catboost import CatBoostRegressor,Pool


#Pipeline
from sklearn.pipeline import Pipeline

#Tuning de Parametros
from sklearn.model_selection import GridSearchCV

/kaggle/input/tabular-playground-series-jan-2021/sample_submission.csv
/kaggle/input/tabular-playground-series-jan-2021/train.csv
/kaggle/input/tabular-playground-series-jan-2021/test.csv


# 2. Dataset de entrenamiento / Dataset de testing

In [2]:
#Dataset con los datos de entrenamiento

train = pd.read_csv(input_path / 'train.csv', index_col='id')
train.head()

,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,target
id,,,,,,,,,,,,,,,
1,0.670390,0.811300,0.643968,0.291791,0.284117,0.855953,0.890700,0.285542,0.558245,0.779418,0.921832,0.866772,0.878733,0.305411,7.243043
3,0.388053,0.621104,0.686102,0.501149,0.643790,0.449805,0.510824,0.580748,0.418335,0.432632,0.439872,0.434971,0.369957,0.369484,8.203331
4,0.834950,0.227436,0.301584,0.293408,0.606839,0.829175,0.506143,0.558771,0.587603,0.823312,0.567007,0.677708,0.882938,0.303047,7.776091
5,0.820708,0.160155,0.546887,0.726104,0.282444,0.785108,0.752758,0.823267,0.574466,0.580843,0.769594,0.818143,0.914281,0.279528,6.957716
8,0.935278,0.421235,0.303801,0.880214,0.665610,0.830131,0.487113,0.604157,0.874658,0.863427,0.983575,0.900464,0.935918,0.435772,7.951046


In [3]:
#Dataset con los datos de testing

test = pd.read_csv(input_path / 'test.csv', index_col='id')
train.head()

,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,target
id,,,,,,,,,,,,,,,
1,0.670390,0.811300,0.643968,0.291791,0.284117,0.855953,0.890700,0.285542,0.558245,0.779418,0.921832,0.866772,0.878733,0.305411,7.243043
3,0.388053,0.621104,0.686102,0.501149,0.643790,0.449805,0.510824,0.580748,0.418335,0.432632,0.439872,0.434971,0.369957,0.369484,8.203331
4,0.834950,0.227436,0.301584,0.293408,0.606839,0.829175,0.506143,0.558771,0.587603,0.823312,0.567007,0.677708,0.882938,0.303047,7.776091
5,0.820708,0.160155,0.546887,0.726104,0.282444,0.785108,0.752758,0.823267,0.574466,0.580843,0.769594,0.818143,0.914281,0.279528,6.957716
8,0.935278,0.421235,0.303801,0.880214,0.665610,0.830131,0.487113,0.604157,0.874658,0.863427,0.983575,0.900464,0.935918,0.435772,7.951046


In [4]:
#Dataset para realizar el submit

submission = pd.read_csv(input_path / 'sample_submission.csv', index_col='id')
display(submission.head())

,target
id,
0,0.5
2,0.5
6,0.5
7,0.5
10,0.5


# 3. Split del dataset train

In [5]:
#Selecciono la columna del dataset que tiene la etiqueta del target

target = train['target']
target.head()

id
1    7.243043
3    8.203331
4    7.776091
5    6.957716
8    7.951046
Name: target, dtype: float64

In [6]:
#Realizo la division de train/test
#Elijo una division 70% / 30%

train_clean = train.drop(columns=['target'])
X_train, X_test, y_train, y_test = train_test_split(train_clean, target, train_size=0.70)

In [7]:
X_train.head()

,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14
id,,,,,,,,,,,,,,
133446,0.476869,0.421288,0.571577,0.482956,0.721616,0.440552,0.568010,0.352912,0.373607,0.407901,0.331498,0.585089,0.371675,0.304354
109273,0.683405,0.489241,0.345634,0.801555,0.283937,0.974009,0.691617,0.899915,0.574206,0.800602,0.750143,0.866188,0.907505,0.381547
214136,0.482417,0.620072,0.715915,0.345733,0.279634,0.771228,0.433654,0.297406,0.439148,0.460909,0.618003,0.700571,0.782484,0.250167
344181,0.635677,0.761387,0.757297,0.717106,0.630509,0.722329,0.458083,0.279417,0.589665,0.764784,0.730516,0.682987,0.503096,0.484154
35566,0.922585,0.619218,0.344768,0.453876,0.285254,0.643101,0.464513,1.004732,0.867902,0.824684,0.828113,0.789784,0.865207,0.753535


In [8]:
X_test.head()

,cont1,cont2,cont3,cont4,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14
id,,,,,,,,,,,,,,
273685,0.371857,0.802084,0.792662,0.290723,0.542846,0.645548,0.558445,0.587706,0.433400,0.523273,0.450642,0.720747,0.731458,0.206403
70108,0.898167,0.419648,0.159063,0.487287,0.609557,0.837811,0.555891,0.518765,0.863220,0.811597,0.707859,0.642789,0.857097,0.236331
380281,0.289344,0.616069,0.615182,0.270571,1.025424,0.401756,0.320786,0.504399,0.112137,0.334169,0.263240,0.301018,0.374867,0.865961
188840,0.536490,0.257646,0.728133,0.307110,0.398636,0.668323,0.833567,0.236559,0.417199,0.708672,0.652415,0.598715,0.690563,0.613574
152015,0.508318,0.080611,0.515954,0.446423,0.750295,0.762214,0.383463,0.357755,0.370746,0.393188,0.496593,0.654606,0.798841,0.626705


In [9]:
y_train.head()

id
133446    6.591225
109273    5.534806
214136    8.454375
344181    9.290339
35566     7.763786
Name: target, dtype: float64

In [10]:
y_test.head()

id
273685    8.171876
70108     7.880259
380281    7.135130
188840    7.983703
152015    8.692724
Name: target, dtype: float64

# 4. Catboost Regresion

## 4.0 Modelo Base

In [11]:
model0 = CatBoostRegressor(iterations=3000)

In [12]:
eval_dataset = Pool(X_test,y_test)

In [13]:
model0.fit(X_train,y_train,use_best_model=True,eval_set=eval_dataset,plot=True)

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

Learning rate set to 0.063847
0:	learn: 0.7318286	test: 0.7318163	best: 0.7318163 (0)	total: 97.6ms	remaining: 4m 52s
1:	learn: 0.7307903	test: 0.7308045	best: 0.7308045 (1)	total: 132ms	remaining: 3m 17s
2:	learn: 0.7298292	test: 0.7298673	best: 0.7298673 (2)	total: 162ms	remaining: 2m 41s
3:	learn: 0.7289466	test: 0.7289802	best: 0.7289802 (3)	total: 191ms	remaining: 2m 22s
4:	learn: 0.7281421	test: 0.7282088	best: 0.7282088 (4)	total: 219ms	remaining: 2m 11s
5:	learn: 0.7274372	test: 0.7274879	best: 0.7274879 (5)	total: 246ms	remaining: 2m 2s
6:	learn: 0.7267815	test: 0.7268525	best: 0.7268525 (6)	total: 275ms	remaining: 1m 57s
7:	learn: 0.7262408	test: 0.7263521	best: 0.7263521 (7)	total: 305ms	remaining: 1m 53s
8:	learn: 0.7256902	test: 0.7258055	best: 0.7258055 (8)	total: 333ms	remaining: 1m 50s
9:	learn: 0.7251967	test: 0.7253290	best: 0.7253290 (9)	total: 364ms	remaining: 1m 48s
10:	learn: 0.7246776	test: 0.7248334	best: 0.7248334 (10)	total: 395ms	remaining: 1m 47s
11:	learn: 

In [14]:
y_predicted0 = model0.predict(X_test)

y_predicted0

array([8.23626294, 7.93403554, 7.87892877, ..., 8.00954906, 8.36705963,
       8.03904229])

In [15]:
score_model0 = mean_squared_error(y_test, y_predicted0, squared=False)

print(f'{score_model0:0.5f}')

0.69799


# 5. Generacion del Submit

## 5.1 Opcion 1

In [16]:
submission['target'] = model0.predict(test)

submission.to_csv('catboost_regression.csv')